In [16]:
import fitz
import numpy as np
import pandas as pd
from tqdm import tqdm

import io
from PIL import Image, ImageDraw, ImageFont, ImageColor

In [17]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [18]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

vol1_index = list(range(616, 638)) #inclusive
vol2_index = list(range(703, 724))
vol3_index = list(range(555, 582))

In [19]:
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

In [20]:
page_num = 616

In [26]:
def epithet_match(row):
    return row['word_num'] == 0 and \
           row['word'].isalpha() and \
           row['word'].islower()

def genus_match(row):
    return row['word_num'] == 0 and \
           row['word'].isalpha() and \
           row['word'][0].isupper() and row['word'][1:].islower()

In [39]:
vol1_char_df

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_ascender,span_descender,span_origin,span_bbox,word_num,word,char_num,char,char_origin,char_bbox
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,0,A,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 119.4..."
1,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,1,U,"(119.42528533935547, 136.92001342773438)","(119.42528533935547, 127.92001342773438, 128.0..."
2,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,2,T,"(127.97057342529297, 136.92001342773438)","(127.97057342529297, 127.92001342773438, 135.2..."
3,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,3,R,"(135.19717407226562, 136.92001342773438)","(135.19717407226562, 127.92001342773438, 143.1..."
4,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,0.75,-0.25,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,4,E,"(143.0890655517578, 136.92001342773438)","(143.0890655517578, 127.92001342773438, 150.34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764639,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,7,E,"(227.60250854492188, 299.47998046875)","(227.60250854492188, 294.22998046875, 232.2247..."
1764640,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,8,-,"(232.03839111328125, 299.47998046875)","(232.03839111328125, 294.22998046875, 234.3461..."
1764641,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,9,S,"(234.15965270996094, 299.47998046875)","(234.15965270996094, 294.22998046875, 238.0137..."
1764642,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,0.75,-0.25,"(134.63999938964844, 299.47998046875)","(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,10,I,"(237.82630920410156, 299.47998046875)","(237.82630920410156, 294.22998046875, 240.5231..."


In [31]:
vol1_char_df[(vol1_char_df['page_num'] == page_num)& (vol1_char_df[(vol1_char_df['page_num'] == page_num)].apply(genus_match, axis = 1))].iloc[:,:-4].drop_duplicates()

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,span_size,span_flags,span_font,span_color,span_ascender,span_descender,span_origin,span_bbox,word_num,word
1715596,1,616,3,5,"(28.559999465942383, 163.30438232421875, 93.46...",0,0,"(1.0, 0.0)","(28.559999465942383, 163.30438232421875, 93.46...",0,8.487576,4,Times-Roman,0,0.959,-0.322,"(28.559999465942383, 171.3599853515625)","(28.559999465942383, 163.30438232421875, 93.46...",0,Acanthophyllum
1715634,1,616,5,8,"(28.799999237060547, 182.2392120361328, 53.568...",0,0,"(1.0, 0.0)","(28.799999237060547, 182.2392120361328, 53.568...",0,8.809273,6,Times-Italic,0,0.962,-0.283,"(28.799999237060547, 190.32000732421875)","(28.799999237060547, 182.2392120361328, 53.568...",0,Aceras
1715714,1,616,10,18,"(28.31999969482422, 230.264404296875, 75.07479...",0,0,"(1.0, 0.0)","(28.31999969482422, 230.264404296875, 75.07479...",0,8.400000,4,Times-Roman,0,0.959,-0.322,"(28.31999969482422, 238.32000732421875)","(28.31999969482422, 230.264404296875, 75.07479...",0,Achyranthes
1715745,1,616,13,24,"(28.31999969482422, 259.0391845703125, 74.4095...",0,0,"(1.0, 0.0)","(28.31999969482422, 259.0391845703125, 74.4095...",0,8.801220,6,Times-Italic,0,0.962,-0.283,"(28.31999969482422, 267.1199951171875)","(28.31999969482422, 259.0391845703125, 74.4095...",0,Acrostichutn
1715778,1,616,16,28,"(28.31999969482422, 287.6243896484375, 66.3888...",0,0,"(1.0, 0.0)","(28.31999969482422, 287.6243896484375, 66.3888...",0,8.400000,4,Times-Roman,0,0.959,-0.322,"(28.31999969482422, 295.67999267578125)","(28.31999969482422, 287.6243896484375, 66.3888...",0,Adiantum
1715804,1,616,18,30,"(28.079999923706055, 306.58441162109375, 60.63...",0,0,"(1.0, 0.0)","(28.079999923706055, 306.58441162109375, 60.63...",0,8.707077,4,Times-Roman,0,0.959,-0.322,"(28.079999923706055, 314.6400146484375)","(28.079999923706055, 306.58441162109375, 60.63...",0,Aegilops
1716201,1,616,31,57,"(26.639999389648438, 555.4644165039062, 76.838...",0,0,"(1.0, 0.0)","(26.639999389648438, 555.4644165039062, 76.838...",0,8.568700,4,Times-Roman,0,0.959,-0.322,"(26.639999389648438, 563.52001953125)","(26.639999389648438, 555.4644165039062, 76.838...",0,Aegilotricum
1716241,1,616,33,59,"(26.399999618530273, 574.6643676757812, 58.346...",0,0,"(1.0, 0.0)","(26.399999618530273, 574.6643676757812, 58.346...",0,8.614977,4,Times-Roman,0,0.959,-0.322,"(26.399999618530273, 582.719970703125)","(26.399999618530273, 574.6643676757812, 58.346...",0,Aellenia
1716511,1,616,50,79,"(234.47999572753906, 131.1444091796875, 273.34...",0,0,"(1.0, 0.0)","(234.47999572753906, 131.1444091796875, 273.34...",0,8.570182,4,Times-Roman,0,0.959,-0.322,"(234.47999572753906, 139.20001220703125)","(234.47999572753906, 131.1444091796875, 273.34...",0,Aeluropus
1716750,1,616,60,99,"(234.24000549316406, 276.58441162109375, 282.0...",0,0,"(1.0, 0.0)","(234.24000549316406, 276.58441162109375, 282.0...",0,8.565611,4,Times-Roman,0,0.959,-0.322,"(234.24000549316406, 284.6400146484375)","(234.24000549316406, 276.58441162109375, 282.0...",0,Agrostemma


In [32]:
vol1_char_df[(vol1_char_df['page_num'] == page_num)& (vol1_char_df[(vol1_char_df['page_num'] == page_num)].apply(epithet_match, axis = 1))].iloc[:,:-4].drop_duplicates()

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,span_size,span_flags,span_font,span_color,span_ascender,span_descender,span_origin,span_bbox,word_num,word
1715580,1,616,2,3,"(42.47999954223633, 153.70440673828125, 114.12...",0,0,"(1.0, 0.0)","(42.47999954223633, 153.70440673828125, 114.12...",0,8.549409,4,Times-Roman,0,0.959,-0.322,"(42.47999954223633, 161.760009765625)","(42.47999954223633, 153.70440673828125, 99.590...",0,cilicica
1715610,1,616,4,7,"(42.2400016784668, 172.90438842773438, 151.875...",0,0,"(1.0, 0.0)","(42.2400016784668, 172.90438842773438, 151.875...",0,8.400000,4,Times-Roman,0,0.959,-0.322,"(42.2400016784668, 180.95999145507812)","(42.2400016784668, 172.90438842773438, 113.440...",0,kurdicum
1715640,1,616,6,10,"(42.2400016784668, 191.83921813964844, 87.3943...",0,0,"(1.0, 0.0)","(42.2400016784668, 191.83921813964844, 87.3943...",0,8.702376,6,Times-Italic,0,0.962,-0.283,"(42.2400016784668, 199.92001342773438)","(42.2400016784668, 191.83921813964844, 64.2381...",0,qffinis
1715653,1,616,7,12,"(42.47999954223633, 201.43919372558594, 135.34...",0,0,"(1.0, 0.0)","(42.47999954223633, 201.43919372558594, 135.34...",0,8.627663,6,Times-Italic,0,0.962,-0.283,"(42.47999954223633, 209.51998901367188)","(42.47999954223633, 201.43919372558594, 94.993...",0,anthropophora
1715675,1,616,8,15,"(42.0, 211.03919982910156, 110.82980346679688,...",0,0,"(1.0, 0.0)","(42.0, 211.03919982910156, 110.82980346679688,...",0,8.754154,6,Times-Italic,0,0.962,-0.283,"(42.0, 219.1199951171875)","(42.0, 211.03919982910156, 68.18370056152344, ...",0,hircina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717111,1,616,75,125,"(246.72000122070312, 526.1591796875, 328.25982...",0,0,"(1.0, 0.0)","(246.72000122070312, 526.1591796875, 328.25982...",0,8.583920,6,Times-Italic,0,0.962,-0.283,"(246.72000122070312, 534.239990234375)","(246.72000122070312, 526.1591796875, 276.69345...",0,insularis
1717146,1,616,77,127,"(246.47999572753906, 554.984375, 289.592407226...",0,0,"(1.0, 0.0)","(246.47999572753906, 554.984375, 289.592407226...",0,8.400000,4,Times-Roman,0,0.959,-0.322,"(246.47999572753906, 563.0399780273438)","(246.47999572753906, 554.984375, 289.592407226...",0,hispanicum
1717162,1,616,79,130,"(246.47999572753906, 574.63916015625, 316.5275...",0,0,"(1.0, 0.0)","(246.47999572753906, 574.63916015625, 316.5275...",0,8.557452,6,Times-Italic,0,0.962,-0.283,"(246.47999572753906, 582.719970703125)","(246.47999572753906, 574.63916015625, 280.0437...",0,arcuatum
1717178,1,616,80,132,"(246.47999572753906, 583.7592163085938, 302.16...",0,0,"(1.0, 0.0)","(246.47999572753906, 583.7592163085938, 302.16...",0,8.663246,6,Times-Italic,0,0.962,-0.283,"(246.47999572753906, 591.8400268554688)","(246.47999572753906, 583.7592163085938, 292.49...",0,damasonium


In [84]:
#rightmost point of any bounding box:
def get_center_x0_INDEXONLY(vol_char_df, page_num):
    df = vol_char_df[vol_char_df['page_num'] == page_num]
    
    right_bound = df['span_bbox'].apply(lambda x : x[2]).max() 
    #leftmost point of any bounding box:
    left_bound = df['span_bbox'].apply(lambda x : x[0]).min()

    return 0.5*(right_bound + left_bound) - 10


def get_col_num(coords, center_x0):
    x0, y0, x1, y1 = coords
    return int(x0 >= center_x0)


all_vol_data = [(vol1_char_df, vol1_index, vol1_doc),
                (vol2_char_df, vol2_index, vol2_doc),
                (vol3_char_df, vol3_index, vol3_doc)]

for vol_char_df, vol_index, doc in all_vol_data: 
    #for each volume 
    for page_num in tqdm(vol_index):
        center_x0 = get_center_x0_INDEXONLY(vol_char_df, page_num)
        vol_char_df['col_num'] = vol_char_df['span_bbox'].apply(lambda coords : get_col_num(coords, center_x0))

100%|██████████| 27/27 [00:14<00:00,  1.90it/s]


In [85]:
#page_num = 616
# vol_char_df = vol3_char_df
# vol_index = vol3_index
# doc = vol3_doc

# image_list = []
# output_name = "strickt_match_vol3"

all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "strickt_match_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "strickt_match_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "strickt_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df[(vol_char_df['page_num'] == page_num)].apply(genus_match, axis = 1))
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df[(vol_char_df['page_num'] == page_num)].apply(epithet_match, axis = 1))
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
        if num_genus_col <= 2:
            print(f"***FLAG***\n number of patterns matched for genus \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")
        if epithet_db.shape[0] <= 2:
            print(f"***FLAG***\n number of patterns matched for epithet \n in page {page_num - vol_index[0] + 1} col {col} is {num_genus_col}")

        #genus is blue, 4
        for coord in genus_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=4)
        #epithet is red, 3
        for coord in epithet_db['span_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

 68%|██████▊   | 15/22 [00:08<00:03,  1.86it/s]

***FLAG***
 number of patterns matched for genus 
 in page 15 col 1 is 2


 91%|█████████ | 20/22 [00:10<00:01,  1.88it/s]

***FLAG***
 number of patterns matched for genus 
 in page 20 col 1 is 0


 19%|█▉        | 4/21 [00:02<00:10,  1.68it/s]

...

 89%|████████▉ | 24/27 [00:12<00:01,  2.01it/s]

***FLAG***
 number of patterns matched for genus 
 in page 24 col 1 is 2


 96%|█████████▋| 26/27 [00:13<00:00,  1.99it/s]

***FLAG***
 number of patterns matched for genus 
 in page 26 col 1 is 0


100%|██████████| 27/27 [00:13<00:00,  1.96it/s]
